In [2]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [3]:
data = sm.datasets.fair.load_pandas().data

In [4]:
data['affair'] = (data.affairs>0).astype(int)

In [5]:
from patsy import dmatrices

In [26]:
y,X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + religious + educ + C(occupation) + C(occupation_husb)',data,return_type="dataframe")

In [27]:
X.head()

,Intercept,C(occupation)[T.2.0],C(occupation)[T.3.0],C(occupation)[T.4.0],C(occupation)[T.5.0],C(occupation)[T.6.0],C(occupation_husb)[T.2.0],C(occupation_husb)[T.3.0],C(occupation_husb)[T.4.0],C(occupation_husb)[T.5.0],C(occupation_husb)[T.6.0],rate_marriage,age,yrs_married,children,religious,educ
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,32.0,9.0,3.0,3.0,17.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,27.0,13.0,3.0,1.0,14.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,22.0,2.5,0.0,1.0,16.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,37.0,16.5,4.0,3.0,16.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,27.0,9.0,1.0,1.0,14.0


In [28]:
X = X.rename(columns={"C(occupation)[T.2.0]":"occ2","C(occupation)[T.3.0]":"occ3","C(occupation)[T.4.0]":"occ4","C(occupation)[T.5.0]": "occ5",
                     "C(occupation)[T.6.0]": "occ6","C(occupation_husb)[T.2.0]":"occ_husb_2","C(occupation_husb)[T.3.0]":"occ_husb_3",
                     "C(occupation_husb)[T.4.0]":"occ_husb_4","C(occupation_husb)[T.5.0]":"occ_husb_5","C(occupation_husb)[T.6.0]":"occ_husb_6"})

In [29]:
X.head()

,Intercept,occ2,occ3,occ4,occ5,occ6,occ_husb_2,occ_husb_3,occ_husb_4,occ_husb_5,occ_husb_6,rate_marriage,age,yrs_married,children,religious,educ
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,32.0,9.0,3.0,3.0,17.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,27.0,13.0,3.0,1.0,14.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,22.0,2.5,0.0,1.0,16.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,37.0,16.5,4.0,3.0,16.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,27.0,9.0,1.0,1.0,14.0


In [30]:
y=np.ravel(y)

In [32]:
len(y)

6366

In [34]:
X.shape

(6366, 17)

In [44]:
data.affair.value_counts(normalize=True)

0    0.677505
1    0.322495
Name: affair, dtype: float64

In [47]:
X["affair"]=y

In [49]:
X.affair.value_counts(normalize=True)

0.0    0.677505
1.0    0.322495
Name: affair, dtype: float64

In [58]:
X.drop('Intercept',axis=1,inplace=True)

In [60]:
X.iloc[:,:16]

,occ2,occ3,occ4,occ5,occ6,occ_husb_2,occ_husb_3,occ_husb_4,occ_husb_5,occ_husb_6,rate_marriage,age,yrs_married,children,religious,educ
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,32.0,9.0,3.0,3.0,17.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,27.0,13.0,3.0,1.0,14.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,22.0,2.5,0.0,1.0,16.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,37.0,16.5,4.0,3.0,16.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,27.0,9.0,1.0,1.0,14.0


In [53]:
X.isna().sum()

Intercept        0
occ2             0
occ3             0
occ4             0
occ5             0
occ6             0
occ_husb_2       0
occ_husb_3       0
occ_husb_4       0
occ_husb_5       0
occ_husb_6       0
rate_marriage    0
age              0
yrs_married      0
children         0
religious        0
educ             0
affair           0
dtype: int64

In [54]:
X.isnull().sum()

Intercept        0
occ2             0
occ3             0
occ4             0
occ5             0
occ6             0
occ_husb_2       0
occ_husb_3       0
occ_husb_4       0
occ_husb_5       0
occ_husb_6       0
rate_marriage    0
age              0
yrs_married      0
children         0
religious        0
educ             0
affair           0
dtype: int64

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:,:16], y, test_size=0.33, random_state=42)

In [62]:
clr = LogisticRegression()

In [64]:
clr.fit(X_train,y_train)

C:\Users\v-avkuma\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [65]:
pred = clr.predict(X_test)

In [66]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [67]:
accuracy_score(y_test,pred)

0.7272727272727273

In [68]:
confusion_matrix(y_test,pred)

array([[1263,  141],
       [ 432,  265]], dtype=int64)

In [74]:
clr.predict_proba(X_test)

array([[0.67565638, 0.32434362],
       [0.51264751, 0.48735249],
       [0.69464773, 0.30535227],
       ...,
       [0.28427024, 0.71572976],
       [0.79927802, 0.20072198],
       [0.62097036, 0.37902964]])